TODO:
- Double check class_mapping.json
- Create necessary hyperparameter constants
- Add references to README

In [3]:
import os
import json
import timm
import torch
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from timm.data import resolve_model_data_config
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Data paths
ORIGINAL_DATA_PATH = "Data/Raw/Images"
SPLIT_DATA_PATH = "Data/Processed"

# 
VALID_SIZE = 0.15
TEST_SIZE = 0.15
NUM_CLASSES = 120

# Model hyperparameters
BATCH_SIZE = 8          # Max 12 
EPOCHS = 20             # 3-5 to train head, 10-15 to train head + two top layers of base model
LEARNING_RATE = 0.001
NUM_WORKERS = 2         # Max 4


In [ ]:
# Returns True if the file at the specified path is an uncorrupted image; False otherwise
def is_image(path):
    try:
        Image.open(path).verify()
        return True
    except Exception as e:
        return False

# Performs a stratified split of the data into training, validation, and test sets
def split_data(input_path, output_path, valid_size=VALID_SIZE, test_size=TEST_SIZE):
    # Clean up any existing data splits
    for dir in [output_path, "Train", "Valid", "Test"]:
        if os.path.exists(dir):
            shutil.rmtree(dir)

    # Create the Train, Valid, and Test directories
    for dir in ["Train", "Valid", "Test"]:
        os.makedirs(os.path.join(output_path, dir), exist_ok=True)

    # 
    breed_dirs = [d for d in os.listdir(input_path) if os.path.isdir(os.path.join(input_path, d))] 
    for breed in breed_dirs:
        # Get all image names for the current breed
        breed_path = os.path.join(input_path, breed)
        images = [f for f in os.listdir(breed_path) if is_image(os.path.join(breed_path, f))]

        # Split the images into (train + validation) and test sets
        train_valid_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=27) 

        # Further split the (train + validation) image set into train and validation sets
        train_imgs, valid_imgs = train_test_split(train_valid_imgs, test_size=valid_size/(1-test_size), random_state=27)

        # Copy images into their appropriate locations
        for set, img_list in zip(['Train', 'Valid', 'Test'], [train_imgs, valid_imgs, test_imgs]):
            set_breed_path = os.path.join(output_path, set, breed)
            os.makedirs(set_breed_path, exist_ok=True)
            for img in img_list:
                shutil.copy(os.path.join(breed_path, img), os.path.join(set_breed_path, img))

split_data(ORIGINAL_DATA_PATH, SPLIT_DATA_PATH, VALID_SIZE, TEST_SIZE)

In [ ]:
# Load the EfficientNet-B3 model, pretrained on ImageNet-1K, and replace its head 
model = timm.create_model(
    "efficientnet_b3",
    pretrained=True,
    num_classes=NUM_CLASSES
)

# Retrieve model configurations to ensure that inputs will adhere to them 
data_config = resolve_model_data_config(model)
img_size = data_config["input_size"][-1]
mean = data_config["mean"]
std = data_config["std"]

In [ ]:
def get_dataloaders(data_dir, img_size=img_size, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    # Transforms
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)), 
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.02
        ), 

        transforms.RandomRotation(degrees=10), 
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])  

    valid_test_transforms = transforms.Compose([
        transforms.Resize(int(img_size * 1.15)),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

    # Image datasets 
    image_datasets = {
        "train":datasets.ImageFolder(root=os.path.join(data_dir, "Train"), transform=train_transforms), 
        "valid":datasets.ImageFolder(root=os.path.join(data_dir, "Valid"), transform=valid_test_transforms), 
        "test":datasets.ImageFolder(root=os.path.join(data_dir, "Test"), transform=valid_test_transforms)
    }

    # DataLoaders
    data_loaders = {
        "train":DataLoader(image_datasets["train"], batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True), 
        "valid":DataLoader(image_datasets["valid"], batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True), 
        "test":DataLoader(image_datasets["test"], batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    }

    return image_datasets, data_loaders

image_datasets, data_loaders = get_dataloaders(SPLIT_DATA_PATH, img_size, BATCH_SIZE, NUM_WORKERS)

Phase 1 — Warmup (frozen backbone)

- Freeze everything except the head.

- Train 3–5 epochs with:

    - lr = 1e-3

    - batch_size = 8

    - mixed precision on MPS

Phase 2 — Minimal unfreeze

- Unfreeze only the last block or last 2 blocks of EfficientNet-B3 + keep the head trainable.

- Use lower LR for those unfrozen backbone layers, e.g.:
    ```python
    optimizer = AdamW(
        [
            {"params": backbone_last_blocks, "lr": 1e-4},
            {"params": head_params, "lr": 5e-4},
        ],
        weight_decay=1e-4,
    )
    ```

- Train 10–15 more epochs.

In [ ]:
# WOOF now i have to actually train this thing. 

# Freeze all layers at first and then perform gradual unfreezing 


# Move the model, labels, and inputs to the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device); # Semicolon to suppress output 